- **필요한 패키지들 import**


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

- **데이터 가져오기 및 확인하기**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
wine = pd.read_csv("/content/drive/My Drive/Colab Notebooks/team project/wine_tokenizing_final.csv")  

In [ ]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  325 non-null    int64 
 1   name_ko     325 non-null    object
 2   blogreview  325 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.7+ KB


In [ ]:
wine.head()

,Unnamed: 0,name_ko,blogreview
0,0,더 롱 리틀 독 스파클링,과일 꽃 아로마 스파클링 좋은 복숭아 입 느껴진다 레몬 산미 살구 기분 컬러 흰 상...
1,1,루베롱 블랑,향 꽃 맛 입 레몬 섬세한 인상 부드럽고 아카시아 헤이즐넛 피니시 노란색 어우러진다...
2,2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",과일 생산 꽃 스파클링 품종 샴페인 감 느낄 그 누 있으며 우아한 뛰어난 균형 이름...
3,3,"마스카 델 타코, 로시 피노 네로 로사토",향 과실 입 미네랄 여운 딸기 그 우아한 달콤한 향신료 부드럽고 풍부하고 옅은 야생...
4,4,"비에티, 바르베라 다스티 '트레 비그네'",향 오크 포도 생산 체리 숙성 지역 느껴진다 붉은 타닌 산도 색 여운 익은 통 입안...


- **필요없는 칼럼 "unnamed: 0"과 중복 와인 지우기**

In [ ]:
del wine['Unnamed: 0']

In [ ]:
wine.head()

,name_ko,blogreview
0,더 롱 리틀 독 스파클링,과일 꽃 아로마 스파클링 좋은 복숭아 입 느껴진다 레몬 산미 살구 기분 컬러 흰 상...
1,루베롱 블랑,향 꽃 맛 입 레몬 섬세한 인상 부드럽고 아카시아 헤이즐넛 피니시 노란색 어우러진다...
2,"마리 스튜아트, 뀌베 드 라 헨느 브뤼",과일 생산 꽃 스파클링 품종 샴페인 감 느낄 그 누 있으며 우아한 뛰어난 균형 이름...
3,"마스카 델 타코, 로시 피노 네로 로사토",향 과실 입 미네랄 여운 딸기 그 우아한 달콤한 향신료 부드럽고 풍부하고 옅은 야생...
4,"비에티, 바르베라 다스티 '트레 비그네'",향 오크 포도 생산 체리 숙성 지역 느껴진다 붉은 타닌 산도 색 여운 익은 통 입안...


In [ ]:
wine = wine.drop_duplicates(subset= 'name_ko')

In [ ]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305 entries, 0 to 324
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name_ko     305 non-null    object
 1   blogreview  305 non-null    object
dtypes: object(2)
memory usage: 7.1+ KB


In [ ]:
wine.isnull().sum()

name_ko       0
blogreview    0
dtype: int64

- **tf-idf 수행하기**

In [ ]:
tfidf = TfidfVectorizer()
# blogreview에 대해 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(wine['blogreview'])
print(tfidf_matrix.shape)

(305, 13466)


이는 325개의 와인을 표현하기 위해 총 13466 개의 단어가 사용되었음을 보여줌.

- **코사인 유사도 구하기**

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

- **와인의 이름과 인덱스를 가진 테이블 만들기**


*  이는 와인 이름을 입력하면 인덱스를 리턴하기 위함 !

In [ ]:
indices = pd.Series(wine.index, index=wine['name_ko'])
print(indices.head())

name_ko
더 롱 리틀 독 스파클링             0
루베롱 블랑                    1
마리 스튜아트, 뀌베 드 라 헨느 브뤼     2
마스카 델 타코, 로시 피노 네로 로사토    3
비에티, 바르베라 다스티 '트레 비그네'    4
dtype: int64


In [ ]:
indices.describe()

count    305.000000
mean     156.400000
std       92.432892
min        0.000000
25%       76.000000
50%      156.000000
75%      235.000000
max      324.000000
dtype: float64

- **get_recommendations 함수 만들기**
* 원하는 특성을 가진 표본을 입력하면 가장 묘사가 유사항 와인 10개를 찾아주는 함수

In [ ]:
def get_recommendations (name_ko, cosine_sim=cosine_sim) :
  # 선택한 와인의 이름으로부터 해당되는 인덱스를 받아옴
  idx = indices[name_ko]
  
  # 모든 와인에 대해 해당 표본과의 유사도를 구함
  sim_scores = list(enumerate(cosine_sim[idx]))

  # 유사도에 따라 와인들 정렬
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  # 가장 유사한 10개의 와인 받아옴
  sim_scores = sim_scores[1:11]

  # 가장 유사한 10개의 와인의 인덱스를 받아옴
  wine_indices = [i[0] for i in sim_scores]

  # 결과 값 리턴
  return wine['name_ko'].iloc[wine_indices]

In [ ]:
get_recommendations('루베롱 블랑')

165    폴 자불레 애네, 꼬뜨 뒤 론 시크렛 드 파미
128                     마레농, 페투라
34             피에르 가이야르, 생 조셉 루즈
7                 이기갈, 꼬뜨 뒤 론 블랑
147               아그리콜라 푸니카, 사마스
142             스칼리올라, 피에몬테 샤르도네
68               루이 로드레, 브륏 프리미에
63            울프 블라스, 골드 라벨 샤도네이
214          발 델레 로제, 리토라레 베르멘티노
15             코노 수르, 20 배럴 샤도네이
Name: name_ko, dtype: object

In [ ]:
wine[wine["name_ko"] == '루베롱 블랑'].blogreview

1    향 꽃 맛 입 레몬 섬세한 인상 부드럽고 아카시아 헤이즐넛 피니시 노란색 어우러진다...
Name: blogreview, dtype: object

In [ ]:
wine[wine["name_ko"] == '마레농, 페투라'].blogreview

128    향 아로마 부드러운 느낄 딸기 매력 입안 있으며 레드 베리 장미 로제 류 향신료 스...
Name: blogreview, dtype: object

In [ ]:
get_recommendations("더 롱 리틀 독 스파클링")

60           카멜 로드 몬테레이 피노 누아
26           루이스 엠 마티니, 나파 밸리
215        비디갈 와이너리, 포르타 6 틴토
251                 크리스마스 아스티
53         코노 수르, 20 배럴 피노 누아
18            투핸즈, 엔젤스 쉐어 쉬라즈
198        제라르 베르트랑, 꼬뜨 로즈 로제
221     보시오, 트로피칼 모스카토 패션 후르츠
5          샹파뉴 끌로드 바롱 사파이어 브뤼
223    얄리, 프리미엄 릴리즈 언오크드 샤르도네
Name: name_ko, dtype: object

In [ ]:
display(wine[wine['name_ko']=='더 롱 리틀 독 스파클링'].blogreview)
display(wine[wine['name_ko']=='카멜 로드 몬테레이 피노 누아'].blogreview)

0    과일 꽃 아로마 스파클링 좋은 복숭아 입 느껴진다 레몬 산미 살구 기분 컬러 흰 상...
Name: blogreview, dtype: object

60    향 과일 체리 꽃 좋은 붉은 산도 느낄 루비 있으며 레드 특징 긴 기분 향기 밝은 ...
Name: blogreview, dtype: object